In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

When we were a statistics student at METU. We chose time series analysis as an elective course. Ceylan Yozgatlıgil was our professor. She gived us a guideline. We copied the step, results and the comments from our R-markdown. This is not a perfect project, but it may help you all :D [http://users.metu.edu.tr/ceylan/STAT%20497.htm]

# STEPS TIME SERİES DATA ANALYSİS
Make sure that you have at least 50 data points in your dataset. Keep last 4 (for yearly and quarterly) or 12 (for monthly) observations as test data. Your data should be up to date. 
1.	Introduction covering data descripition, aim and the source of data.
2.	Time series plot and interpretation (Visually determine the existence of a trend, seasonality, outliers).
3.	Keep several observations out of the analysis to use them to measure the forecast accuracy of the models. (For yearly data and quarterly data 4 or 5, monthly data 12).
4.	Box-Cox transformation analysis: If the series need any transformation, do it. If the information criterion values are too close to each other, don’t transform the data.
5.	Make a anomaly detection and if necessary clean the series from anomalies (use anomalize, forecast (tsclean function) or AnomalyDetection packages).
6.	ACF, PACF plots, KPSS and ADF or PP test results  for zero mean, mean and trend cases and their interpretation. For seasonal unit root, HEGY and OCSB or Canova-Hansen tests are required.
7.	If there is a trend, remove it either by detrending or differencing. You may need to apply unit root tests again.
8.	Then, look at the time series plot of a stationary series, ACF and PACF plots, information table, ESACF.
9.	Identify a proper ARMA or ARIMA model or SARIMA model.
10.	After deciding the order of the possible model (s), run MLE or conditional or uncondinitional LSE and estimate the parameters. Compare the information criteria of several models. (Note: If there is a convergence problem, you can change your estimation method).
11.	Diagnostic Checking: 
a)	On the residuals, perform portmanteau lack of fit test, look at the ACF-PACF plots of the resuduals (for all time points, ACF and PACF values should be in the white noise bands), look at the standardized residuals vs time plot to see any outliers or pattern. 
b)	Use histogram, QQ-plot and Shapiro-Wilk test (in ts analysis, economists prefer Jarque-Bera test) to check normality of residuals. 
c)	Perform Breusch-Godfrey test for possible autocorrelation in residual series. The result should be insignificant.
d)	For the Heteroscedasticity, look at the ACF-PACF plots of the squared residuals (there should be no significant spikes); perform ARCH Engle's Test for Residual Heteroscedasticity under aTSA package. The result should be insignificant. If the result is significant, you can state that the error variance is not constant and it should be modelled, but don’t intend to model the variance. If there is a heteroscedasticity problem, most probably normality test on residuals will fail. The high values in the lower and upper extremes destroy the normality due to high variation. In your project, you can state these only. When solving a real life problem, you cannot just state and quit dealing this problem!
12.	Forecasting: The number of forecasts should be same as the length of your test data.
a.	Perform Minimum MSE Forecast for the stochastic models (like ARIMA or SARIMA) 
b.	Use ets code under the forecast package to choose the best exponential smoothing (simple, Holt’s, Holt-Winter’s) method that suits your series for deterministic forecasting. 
c.	Obtain forecasts using neural networks (nnetar)
13.	If you transformed the series for SARIMA model, back transform the series to reach the estimates for the original units. 
14.	Calculate the forecast accuracy measures and state which model gives the highest performance for your dataset.
15.	Provide plots of the original time series, predictions, forecasts and prediction intervals on the same plot drawing the forecast origin for ARIMA models, exponential smoothing method and neural networks. The plot for each model should look like the following plot.
16.CONCLUSION

In [ ]:
PBNRESCONS <- ts(read.csv("../input/pbnrescons/PBNRESCONS.csv"),start = 2002,frequency = 12)
data<-ts(PBNRESCONS[,2],start=2002,frequency = 12)


The project dataset is downloaded from fred.stlouisfed.org. This data provides information about monthly estimates of the total dollar value of construction in the U.S from 2002 to 2019. As cited in Census.gov, data estimates include the cost of labor and materials, cost of architectural and engineering work, overhead costs, interest and taxes paid during construction, and contractor’s profits. A million dollars define units of the value of spending. The project aims that develop a time series analysis using total public construction spendings information and offer an efficient model to use for generating future expenses.

> Before starting any analysis, it is better to look at the plot of the data.

In [ ]:
par(mfrow=c(3,1))
plot(data)
acf(as.numeric(data))
pacf(as.numeric(data))
#plot shows an increasing trend. ACF shows slow decay that is an indication of being non-stationarity. Also, we know that since our data set is monthly, it is seasonal.

By looking at this plot, it can be said that the data has an increasing trend until 2010. However, between 2010-2015 there are factors that cause decline in trend. Later, there is also an increasing trend. It is also better to look at ACF and PACF plots.

ACF plot shows slow decay that is an indication of being non-stationarity. Non-stationarity means that the mean and the variance of the observations change over time. Also, it is known that the series is seasonal because the data set is monthly.
Firstly, it is necessary to divide data into two as train and test to measure the forecast accuracy of the models.



In [ ]:
data_train=window(data,start = 2002,end=c(2018,9),frequency = 12)
data_test=window(data,start=c(2018,10),end=c(2019,9),frequency=12)
par(mfrow=c(3,1))
plot(data_train)
acf(as.numeric(data_train))
pacf(as.numeric(data_train))
#plot shows an increasing trend. ACF shows slow decay that is an indication of being non-stationarity. Also, we know that since our data set is monthly, it is seasonal.

Secondly, Box-Cox transformation should be done to stabilize variance.

In [ ]:
library(forecast)
lambda <- BoxCox.lambda(data_train)
lambda # lambda is not equal to one, so we need to use transformed data
train_t=BoxCox(data_train,lambda)
par(mfrow=c(3,1))#After transforming the data, the plot still shows an increasing trend. 
plot(train_t)
acf(as.numeric(train_t))#ACF shows slow decay that is an indication of being non-stationarity.
pacf(as.numeric(train_t))


In order to detect unusual observations, anomaly detection is used. Furthermore, one observation is detected as an anomaly. After that this anomaly observation is cleaned from data.

In [ ]:
library(chron)
time=as.chron(train_t)
time
time1=as.Date(time,format="%d-%b-%y") #format is important.
time1
#After this we convert our train data which is ts object into data frame.
train_bc_anomaly=data.frame(trainbc=train_t)
head(train_bc_anomaly)
#Then, add time1 object as rownames to data frame created for anomaly detection.
rownames(train_bc_anomaly)=time1
head(train_bc_anomaly)
library(anomalize) #tidy anomaly detection
library(tidyverse) #tidyverse packages like dplyr, ggplot, tidyr
train_bc_anomaly_ts <- train_bc_anomaly %>% rownames_to_column() %>% as.tibble() %>% 
  mutate(date = as.Date(rowname)) %>% select(-one_of('rowname'))
head(train_bc_anomaly_ts)
#The dataset is prepared for the anomaly detection.
train_bc_anomaly_ts %>% 
  time_decompose(trainbc, method = "stl", frequency = "auto", trend = "auto") %>%
  anomalize(remainder, method = "gesd", alpha = 0.05, max_anoms = 0.2) %>%
  plot_anomaly_decomposition()
train_bc_clean=tsclean(train_t)


In order to detect unusual observations, anomaly detection is used. Furthermore, one observation is detected as an anomaly. After that this anomaly observation is cleaned from data.

In [ ]:
par(mfrow=c(3,1))
plot(train_t)
acf(as.numeric(train_t))
pacf(as.numeric(train_t))

#I should test whether it has unit root or not.
#KPSS
library(tseries)
kpss.test(train_bc_clean,null=c("Level"))
#H0:stationary.Since p-value is less than 0.05, we reject H0 and we can say that the series is not stationary
kpss.test(train_bc_clean,null=c("Trend"))
#H0: There is a deterministic trend. H1: There is a stochastic trend.Since p-value is less than 0.05, we reject H0 and we can say that there is a stochastic trend.
#ADF
library(fUnitRoots)
adfTest(train_bc_clean, lags=1, type="c") #I select lag 1 because of PACF. In PACF only 1st lag is significant. #Because time series plot of the process shows that the mean of the system is not 0 or close to 0. In such cases, we will prefer to use this test with c.
#Ho: The process has unit root (non-stationary). H1: The process is stationary.Since p value is greater than 0.05, we fail to reject H0. It means that we don’t have enough evidence to claim that we have a stationary system.
adfTest(train_bc_clean, lags=1, type="ct") #"ct" is prefered because intercept and time trend exist.
#Our process has unit root.According to ADF, our process has unit root. 
#As a result, It means we have a non-stationary series with stochastic trend. To remove the trend, we apply differencing using diff function later.
library(tseries)
pp.test(train_bc_clean)
#Ho: The process has unit root (non-stationary). H1: The process is stationary.Since p value is greater than 0.05, we fail to reject H0. It means that we don’t have enough evidence to claim that we have a stationary system.
#Kpss, ADF and PP Tests show that there is a unit root problem in the system.
#To solve this,take the difference.

#HEGY Test
library(uroot)
hegy.test(train_bc_clean,deterministic = c(1,1,1),maxlag=12)
#In this output, we will use p value of t_1 for regular unit root and use the p value of F_11:12 for testing seasonal unit root.
#The output shows that the system does not have regular unit roota nd seasonal unit root because p value of t_1 is less than 0.05 and f_11:12 is less than 0.05
#Canova-Hensen Test
ch.test(train_bc_clean,type = "dummy",sid=c(1:12))
#according to Canova Hensen test,we do not have a seasonal unit root problem

After anomaly detection and cleaning data process, the system still looks non-stationary. For this reason, it is better to look at KPSS test.
H0: The system is stationary. H1: The system is non-stationary.
It can said that the system is non-stationary since p-value is less 0.05. Now, it can be looked at trend of the data.


*KPSS test for trend stationarity
H0: There is a deterministic trend. H1: There is a stochastic trend.
Since p-value is less than 0.05, we reject H0, and we can say that there is a stochastic trend.
ADF test and PP test are used to check unit root problems.
Ho: The process has unit root (non-stationary). H1: The process is stationary.
Since p value is greater than 0.05, we fail to reject H0. It means that we don’t have enough evidence to claim that we have a stationary system.
Ho: The process has unit root (non-stationary). H1: The process is stationary.
Since p value is greater than 0.05, we fail to reject H0. It means that we don’t have enough evidence to claim that we have a stationary system.
KPSS, ADF and PP Tests show that there is a unit root problem in the system.
For seasonal unit root, HEGY and Canova-Hansen tests are used. The result below is from HEGY test.



In [ ]:
library(forecast)
ndiffs(train_bc_clean)
dtrain=diff(train_bc_clean) #regular difference
nsdiffs(diff(train_bc_clean))#no need to seasonal difference
plot(dtrain,main="Time Series Plot of the Differenced Data",col="red")
# seems stationary but let's check it with kpss test
kpss.test(dtrain,null=c("Level"))
#Since the p-value is greater than 0.05,the process is stationary
hegy.test(dtrain)
#there is no regular unit root and no seasonal unit root


In this output, p value of tpi_1 gives the result for regular unit root and the p value of Fpi_11:12 gives the result for testing seasonal unit root.
The output shows that the system has both regular and seasonal unit roots because both of the p-values of tpi_1 and Fpi_11:12 are greater than 0.05.
Canova and Hansen (CH) test statistic for the null hypothesis of a stable seasonal pattern.
The output above shows that according to Canova-Hansen test, the seasonal pattern is stable.
In order to remove the trend, differencing is necessary. Then, regular differencing is taken. Taking regular difference did not solved the regular unit root. (p-value of tpi_1 is greater than 0.05) Therefore, seasonal differencing is also taken. 
The process seems stationary around mean 0 after taking differences. But, it is better to check this with KPSS test.
Since the p-value is greater than 0.05,the process is stationary. 
Taking both regular and seasonal differences solved both regular and seasonal unit root problems (p-values of tpi_1 and Fpi_11:12 are less than 0.05)

In [ ]:
par(mfrow=c(2,1))
acf(as.vector(dtrain),main="ACF Of Differenced Data",col="red",lag.max = 36)
pacf(as.vector(dtrain),main="PACF Of  Differenced Data",col="red",lag.max = 36)
#By looking at ACF and PACF plots,we can suggest SARIMA(0,1,0)(2,0,2)[12]

The model SARIMA(2,1,2)(2,1,2)[12] can be suggested by looking at ACF and PACF graphs of differenced data. In addition, from auto.arima the model SARIMA(1,1,1)(2,0,2)[12] can be suggested. Now, it is time to check significance of these models.To do so, we will look at the last estimated parameters of each component. If the ratio between these estimates and their standard errors (s.e) are greater than +2 or less than -2 ,it can said that these parameters are significant and the model is significant.

In [ ]:
install.packages('TSA')
library(TSA)
#ESACF Table
eacf(dtrain)
#From ESACF table,we can suggest ARIMA(3,1,3)

#SARIMA(0,1,0)(2,0,2)[12]
#ARIMA(3,1,1)
#ARIMA(2,1,0)
#SARIMA(1,1,1)(2,0,2)[12]

In [ ]:
install.packages('caschrono')
library(caschrono)
armaselect(dtrain)#MINIC Table
#From MINIC table,we can suggest ARIMA(2,1,0)

In [ ]:
library(forecast)
auto.arima(train_bc_clean)
#ARIMA(1,1,1)(2,0,2)[12]

So, this model is not significant.

In [ ]:
fit1<-Arima(train_bc_clean,order = c(0, 1, 0),seasonal=list(order=c(2,0,2),period=12))
summary(fit1)
ratio_sar2=-0.1632/0.3393
ratio_sar2
ratio_sma2=-0.0815/0.3395
ratio_sma2
#the model is insignificant

In [ ]:
fit2<-Arima(train_bc_clean,order = c(3, 1, 3))
summary(fit2)
ratio_ar3=-0.1609/0.3899
ratio_ar3
ratio_ma3=-0.0318/0.4291
ratio_ma3
#Since both AR3 and MA1 are in the inerval (-2,2),the model is insignificant

In [ ]:
fit3<-Arima(train_bc_clean,order = c(2, 1, 0))
summary(fit3)
ratio_ar2=-0.1066/0.0721
ratio_ar2
#the model is insignificant

In [ ]:
fit4<-Arima(train_bc_clean,order = c(1, 1, 1),seasonal=list(order=c(2,0,2),period=12))
summary(fit4)
ratio_ar1=-0.0525/0.0046
ratio_ar1
ratio_ma1=0.0276/ 0.0131
ratio_ma1
ratio_sar2=-0.2488/0.0100
ratio_sar2
ratio_sma2=0.0743/0.0079
ratio_sma2
#When each parameter estimates is divided by their standard errors,it is understood that the approximated test statistics are greater than 2 for all parameters. 
#That's why, it is said that the model is significant

Since only SARIMA(1,1,1)(2,0,2)[12] model is significant, we continue diagnoctic check with this model.

In [ ]:
plot(y=resid(fit4),x=as.vector(time(data_train)), ylab='Residuals',xlab='Time',type='o')
abline(h=0)
#It seems that the residuals seems stationary around zero. 
par(mfrow=c(1,2))
r=resid(fit4)
acf(r)
pacf(r)
#standardized residuals vs time
Standard.res=rstandard(fit4)
plot(y=Standard.res,x=as.vector(time(data_train)), ylab='Standardized Residuals',xlab='Time',type='o')
#They are scattered around zero and it can be interpreted as zero mean.


1. Since only SARIMA(1,1,1)(2,0,2)[12] model is significant, we continue diagnostic check with this model. Let us look at ACF and PACF plots of residuals.
2. For all time points, ACF and PACF values should be in the white noise bands. However, some of the spikes are outside of the bands. This means there might be some outliers.
3. The standardized residual vs time plot also shows some outliers.

#Checking the normality of the residual

In [ ]:
#QQ Plot
qqnorm(r)
qqline(r)
#Since the points fall aproximately along the qq-line, we can say that residuals are normally distributed

#Histogram
hist(r,main="Histogram of Residuals",breaks=20)
#The histogram of residual shows that they might have a normal distribution with outliers.

#Shapiro-Wilk test & Jarque-Bera test
#Ho: Residuals have normal distribution.
#H1: Residuals don't have normal distribution.
shapiro.test(r)
library(tseries)
jarque.bera.test(r)
#p-values are grater than 0.05, so fail to reject H0.
#We have enough evidence to claim that residuals have normal distribution.

* By looking at Normal Q-Q Plot, it can be said that residuals are normally distributed.
* The histogram of residuals shows that they might have a normal distribution with outliers.
* Ho: Residuals have normal distribution.H1: Residuals do not have normal distribution.
* p-value is greater than 0.05, so fail to reject H0. We don’t have enough evidence to claim that residuals don’t have normal distribution.


#Checking Autocorrelation of Residuals

In [ ]:
#Breusch-Godfrey Test
library(TSA)
library(lmtest)
m = lm(r ~ 1+zlag(r))
bgtest(m,order=15)
#Since p value is greater than α=0.05, we have 95% confident that the residuals of the model are uncorrelated, according to results of Breusch-Godfrey Test.

Since p value is greater than α, we are 95% confident that the residuals of the model are uncorrelated, according to results of Box-Ljung Test.

For the heteroscedasticity, the ACF-PACF plots of the squared residuals should be looked.


In [ ]:
#Check Heteroscedasticity
rr=r^2
par(mfrow=c(1,2))
acf(as.vector(rr),main="ACF of Squared Residuals") 
pacf(as.vector(rr),main="PACF of Squared Residuals")
#Both plots shows that there is no spike out of the white noise bands that is an indication of homoscedasticity

#Breusch-Pagan
m = lm(r ~ data_train+zlag(data_train)+zlag(data_train,2))
bptest(m)

#White Test
m = lm(r ~ data_train+zlag(data_train)+zlag(data_train,2)+zlag(data_train)^2+zlag(data_train,2)^2+zlag(data_train)*zlag(data_train,2))
bptest(m)
#All tests show that there is no heteroscedasticity problem.

Both plots shows that there is no spike out of the white noise bands that is an indication of homoscedasticity.

In [ ]:

archTest(r)

It is homoscedastic, according to archTest. All tests show that there is no heteroscedasticity problem.
As a result, all of the assumptions checked.

Forecasts are obtained from SARIMA(1,1,1)(2,0,2)[12], Holt’s exponential smoothing method, and neural networks (nnetar). 


In [ ]:
#arimafit
forecast1=forecast(fit4,h=12)
forecast1
plot(forecast1)

In [ ]:
#ets
model2<-ets(train_t,model="ZZZ")
forecast2=forecast(model2,h=12)
plot(forecast2)

In [ ]:
#nnetar
model3=nnetar(train_t)
forecast3<-forecast(model3,h=12,PI=T)
plot(forecast3)

Since at the beginning the series is transformed for SARIMA model, back transform is done the series to reach the estimates for the original units. 
At the end,the forecast accuracy measures are calculated for each model

In [ ]:
forecast_for_sarima_t=InvBoxCox(as.numeric(forecast1$mean),lambda=lambda)
forecast_for_ets_t=InvBoxCox(as.numeric(forecast2$mean),lambda)
forecast_for_nnar_t=InvBoxCox(as.numeric(forecast3$mean),lambda)

In [ ]:
accuracy(forecast_for_sarima_t,data_test)#Accuracy for SARIMA
accuracy(forecast_for_ets_t,data_test)#Accuracy for ETS
accuracy(forecast_for_nnar_t,data_test)#Accuracy for NNAR
#MApe value of ETS is the lowest,so it is the best technique to forecast

In [ ]:
#For SARIMA
plot(forecast1)
lines(fit4$fitted,col="red",lty=3)
legend("bottomright",c("Actual","Fitted"),col=c("black","red"),lty = c(1,3))
#For ETS
plot(forecast2)
lines(model2$fitted,col="red",lty=3)
legend("bottomright",c("Actual","Fitted"),col=c("black","red"),lty = c(1,3))
#For NNAR
plot(forecast3)
lines(model3$fitted,col="red",lty=3)
legend("bottomright",c("Actual","Fitted"),col=c("black","red"),lty = c(1,3))

ETS is the best technique to forecast since its MAPE value is the lowest

# CONCLUSION


In conclusion, firstly the time series is plotted to understand behaviour. And it is found that series is non-stationary and follows an increasing trend. Then, data is splitted into two parts in order to measure accuracy of forecasts. Box Cox transformation is applied to stabilize variance after that. Later on, anomaly detection is made and if the series is cleaned from anomalies. KPSS test shows that the system is non-stationary and follows stochastic trend. Moreover, ADF and PP tests also show the series is non-stationary(unit root). For unit roots, HEGY test is used and found that there are both regular and seasonal unit roots.By taking both regular and seasonal differences are solved both regular and seasonal unit root problems. The series also became stationary. Then, by looking ACF and PACF plots model is suggested. Also, from auto.arima function a model is suggested. These models are SARIMA(2,1,2)(2,1,2)[12] and SARIMA(1,1,1)(2,0,2)[12].But only SARIMA(1,1,1)(2,0,2)[12] is significant. Moreover, diagnostics are checked for this model and all of the assumptions are checked.
Secondly, forecasts are obtained with SARIMA model, ETS and neural networks. And back transformation is done because at first Box-Cox transformation is applied. Accuracy measurements are done for forecasts. Moreover, it is found that ETS is the best technique to forecast since its MAPE value is the lowest.

